### 0. Installing Libraries

In [ ]:
# keras and tensorflow version might be wrong, so we need to down/upgrade
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

shell_call("pip install tensorflow==2.10.0 tensorflow-probability==0.16 keras==2.10.0")

In [ ]:
if(!require(deepregression)){
  install.packages("deepregression")
  library(deepregression)
}
# note: this also loads tensorflow and keras (both required in the folllowing)

if (!require("imager")) {
  install.packages("imager")
  library(imager)
}

### 1. Auto Differentiation using TensorFlow

First, a more simple example where we automatically calculate $\frac{\partial f(x)}{\partial x}$ where $f(x) = x^2$.

In [ ]:
# Define a quadratic function f(x) = x^2
# (or any other that is differentiable)
f <- function(x) {
  x^2
}

# Create a scalar variable
x <- tf$Variable(3.0)

# Record the operations to compute the gradient
with(tf$GradientTape() %as% tape, {
  y <- f(x)
})

# Compute the gradient of y with respect to x
grad <- tape$gradient(y, x) # i.e. dy/dx

# Output the gradient -> should be 2*x = 2*3 = 6
print(grad$numpy())

Let's try a bit more complicated function and see if it still works...

In [ ]:
f <- function(X) {
  tf$sqrt(tf$reduce_sum(tf$sinh(X)^2))
}

# Create a 3D variable tensor (an array)
X <- tf$Variable(array(c(1,2,3,4,5,6), dim=c(2,1,3)))

# Record the operations with GradientTape
with(tf$GradientTape() %as% tape, {
  z <- f(X)
})

# Compute gradient
grad_X <- tape$gradient(z, X)

# Print the original 3D tensor and its gradient
cat('Original 3D tensor X:\n')
print(X$numpy())
cat('\nGradient of f w.r.t. X:\n')
print(grad_X$numpy())

So we just computed a derivative w.r.t. an array (!) with no effort.

### 2. Example for Gradient Descent Optimization using the Gradient Tape

In [ ]:
library(tensorflow)

# Define a quadratic function f(x) = x^2
f <- function(theta) {
  theta^2
}

# Create a scalar variable
theta <- tf$Variable(5.0)

# Define a learning rate
lr <- 0.1

# Perform gradient descent
for (i in 1:10) {
  with(tf$GradientTape() %as% tape, {
    y <- f(theta)
  })

  # Compute the gradient of y with respect to theta
  grad <- tape$gradient(y, theta)

  # Update theta: theta <- theta - lr * grad
  theta$assign_sub(lr * grad)

  # Print the current theta and y
  cat('Step', i, ':', 'theta =', theta$numpy(), ', y =', y$numpy(), '\n')
}


### 3. Optimizing Neural Networks in `keras`

- We start with a Multi-layer perceptron (a network with fully connected layers)
- We use the MNIST data as an example, which we first load (incl. in keras)


In [ ]:
# Load the MNIST dataset
mnist <- dataset_mnist()
x_train <- mnist$train$x
y_train <- mnist$train$y
x_test <- mnist$test$x
y_test <- mnist$test$y

We can visualize the data e.g. as follows:

In [ ]:
# Choose the first image
img <- x_train[1,,]

# Visualize the image
image(t(apply(img, 2, rev)), col = grey.colors(255), axes = FALSE)

As we have not yet learned the details of convolutional layers, we start by
converting the images into long vectors (i.e. we use every pixel as a feature)

In [ ]:
# Preprocess the data
x_train <- array_reshape(x_train, c(nrow(x_train), 784)) / 255
x_test <- array_reshape(x_test, c(nrow(x_test), 784)) / 255
y_train <- to_categorical(y_train, 10)
y_test <- to_categorical(y_test, 10)

We can now define the network and train it using the `keras` functionalities:
- `keras_model_sequential`: sets up a very simply sequential model (i.e. one layers follows the other)
- `compile`: compiles / initializes the model (computes gradients etc.)
- `fit`: runs the optimization (loop over epochs, batches and update weights using the optimizer)

In [ ]:
# Define MLP model
model_mlp <- keras_model_sequential() %>%
  layer_dense(units = 256, activation = 'relu', input_shape = c(784)) %>%
  layer_dropout(rate = 0.4) %>%
  layer_dense(units = 128, activation = 'relu') %>%
  layer_dropout(rate = 0.3) %>%
  layer_dense(units = 10, activation = 'softmax')

# Compile the model
model_mlp %>% compile(
  loss = 'categorical_crossentropy',
  optimizer = optimizer_adam(),
  metrics = c('accuracy')
)

# Train the model
history_mlp <- model_mlp %>% fit(
  x_train, y_train, # the data required
  epochs = 10, # only run for 10 epochs -- probably not enough
  batch_size = 128, # batch size
  validation_split = 0.2, # data percentage to validate model performance
  verbose = FALSE # set to TRUE to see details
)

We can visualize the optimization process as follows:

In [ ]:
plot(history_mlp)

We can also evaluate the performance on an independent data set

In [ ]:
# Evaluate performance
model_mlp %>% evaluate(x_test, y_test)

### 4. Optimizing Statistical Models in Neural Networks

Let's try to fit a logistic regression in a neural network and compare with `glm`

In [ ]:
# Load dataset from UCI Machine Learning Repository
url <- "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
data <- read.csv(url, header = FALSE)
data[,1:8] <- lapply(data[,1:8], scale)

# Rename columns
names(data) <- c("Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
                 "BMI", "DiabetesPedigreeFunction", "Age", "Outcome")

# Using glm for logistic regression
glm_model <- glm(Outcome ~ ., family = binomial(), data = data)

# Predicting and evaluating
probabilities_glm <- predict(glm_model, data, type = "response")
coefs_glm <- coef(glm_model)


In [ ]:
# Preprocessing
x <- as.matrix(data[,1:8])
y <- data$Outcome

# Define a single-layer neural network
model <- keras_model_sequential() %>%
  layer_dense(units = 1, activation = "sigmoid", input_shape = ncol(x))

# Compile the model
model %>% compile(
  optimizer = optimizer_rmsprop(),
  loss = "binary_crossentropy",
  metrics = c("accuracy")
)

# Train the model
model %>% fit(
  x, y,
  epochs = 200,
  batch_size = 16,
  validation_split = 0.2,
  callbacks = list(
    # this callback stops as soon as there
    # is no improvement for 10 iterations
    callback_early_stopping(patience = 10,
    restore_best_weights = TRUE)
  )
)

# Predicting and evaluating
probabilities_nn <- model %>% predict(x)
coefs_nn <- model$weights

Let's first compare the predictions

In [ ]:
plot(probabilities_glm ~ as.vector(probabilities_nn))

... and compare estimated coefficients

In [ ]:
rbind(
  coefs_glm,
  coefs_nn = c(unlist(lapply(coefs_nn, as.matrix)[c(2,1)]))
)


### 5. Semi-Structured Networks

Load the AirBnB data

In [ ]:
url <- "https://github.com/davidruegamer/airbnb/raw/main/munich_clean_text.RDS"
destfile <- file.path(getwd(), "munich.RDS")
download.file(url, destfile, mode = "wb")
airbnb <- readRDS("munich.RDS")
airbnb$days_since_last_review <- as.numeric(difftime(airbnb$date, airbnb$last_review))

Check if everything worked as intended...

In [ ]:
str(airbnb,1)

Let's first fit a generalized additive model using in a neural network

In [ ]:
mod_airbnb <- deepregression(y = airbnb$price,
                             list_of_formulas = list(
                               ~ bedrooms + room_type +
                                 s(review_scores_rating),
                               ~ 1
                             ),
                             data = airbnb)
mod_airbnb %>% fit(epochs = 1000,
                   early_stopping = TRUE,
                   patience = 50,
                   verbose = FALSE)

We can visualize the results by, e.g., calling the `plot` function:

In [ ]:
plot(mod_airbnb)

or looking at the estimated coefficients

In [ ]:
coef(mod_airbnb)

Now let's process
- load and pre-process images
- and define suitable models

for defining the semi-structured network that combines the GAM and a convolutional neural network

In [ ]:
## pictures
url1 <- "https://github.com/davidruegamer/deepregression_tutorial/raw/main/32_1.zip"
destfile <- file.path(getwd(), "32_1.zip")
download.file(url1, destfile, mode = "wb")
unzip("32_1.zip")
url2 <- "https://github.com/davidruegamer/deepregression_tutorial/raw/main/32_2.zip"
destfile <- file.path(getwd(), "32_2.zip")
download.file(url2, destfile, mode = "wb")
unzip("32_2.zip")

In [ ]:
airbnb$image <- paste0(getwd(), "/32/", airbnb$id, ".jpg")

The following code defines a simple convolutional neural network (CNN).
We will talk about some of the details later. For now you can just use this as is.

In [ ]:
## -----------------------------------------------------------------------------------------------------
cnn_block <- function(filters, kernel_size, pool_size, rate, input_shape = NULL){
    function(x){
      x %>%
        layer_conv_2d(filters, kernel_size, padding = "same", input_shape = input_shape) %>%
        layer_activation(activation = "relu") %>%
        layer_batch_normalization() %>%
        layer_max_pooling_2d(pool_size = pool_size) %>%
        layer_dropout(rate = rate)
    }
  }


## -----------------------------------------------------------------------------------------------------
cnn <- cnn_block(filters = 16, kernel_size = c(3,3), pool_size = c(3,3), rate = 0.25,
                 shape(200, 200, 3))
deep_model_cnn <- function(x){
    x %>%
    cnn() %>%
    layer_flatten() %>%
    layer_dense(32) %>%
    layer_activation(activation = "relu") %>%
    layer_batch_normalization() %>%
    layer_dropout(rate = 0.5) %>%
    layer_dense(1)
}


Now we define the semi-structured model using the previous deep network

In [ ]:
mod_cnn <- deepregression(
  y = log(airbnb$price),
  list_of_formulas = list(
    ~1 + te(latitude, longitude) + # a spatial effect
      deep_model_cnn(image), # a CNN for images
    ~1),
  data = airbnb,
  list_of_deep_models = list(deep_model_cnn =
    list(deep_model_cnn, c(200,200,3))),
  optimizer = optimizer_adam(lr = 0.0001)
)

In [ ]:
# fitting a deepregression model is similar to keras,
# but with some more arguments
mod_cnn %>% fit(
  epochs = 2, # a small number for demonstration purposes
  early_stopping = TRUE, # directly activate early stopping
  patience = 5, # and set the patience
  verbose = FALSE
)

We can now, e.g., check the spatial effect (result is not really meaningful though)

In [ ]:
plot(mod_cnn)

Or make predictions for some pictures:

In [ ]:
image <- load.image(paste0(getwd(), "/32/", airbnb$id[1], ".jpg"))
plot(image)

In [ ]:
# Make the corresponding prediction
summary(log(airbnb$price))
mod_cnn %>% predict(lapply(airbnb[c("image", "latitude", "longitude")],
                    "[[", 1))

In [ ]:
plot(mod_cnn %>% fitted() ~ log(airbnb$price))

### 6. Convolutional Layers

Let's implement the Sobel filter (only it's horizontal part for demonstration) in a keras network!

In [ ]:
# Define the Sobel filter kernel for horizontal edge detection
sobel_horizontal <- array(c(-1, 0, 1,
                            -2, 0, 2,
                            -1, 0, 1), dim = c(3, 3, 1, 1))

# Building the model
model <- keras_model_sequential() %>%
  layer_conv_2d(filters = 1, kernel_size = c(3,3), input_shape = c(32,32,1),
                use_bias = FALSE, activation = "relu",
                # plug in the filter values -> normally we would learn these
                # in a data-driven fashion!
                kernel_initializer = initializer_constant(
                  value = sobel_horizontal)
               )

Let's apply the model / filter to an image and see the result:

In [ ]:
# Load some examplary data (CIFAR10)
cifar10 <- dataset_cifar10()
x_train <- cifar10$train$x

# Take one image
img <- x_train[2,,,]
img <- array_reshape(img, c(32,32,3))

In [ ]:
# Convert the image to grayscale using imager
img_imager <- as.cimg(img)
img_gray <- grayscale(img_imager)

# Apply the filter to the image
edge_detected_image <- model %>% predict(
  array_reshape(img_gray/255, c(1, 32, 32, 1)))

# Plotting using imager
layout(matrix(1:3, nrow = 1))
plot(img_imager, axes=FALSE, main="Original Image")
plot(img_gray, axes=FALSE, main="Gray Image")

# Convert the prediction result to an imager image object for plotting
edge_detected_imager <- as.cimg(edge_detected_image[1,,,])

# Plotting the edge-detected image using imager
plot(edge_detected_imager, axes=FALSE, main="Edge Detected Image")

### 7. Recurrent Neural Networks

Let's simulate a small time series and try to predict the next value using a seq-to-one RNN network!

First, generate some synthetic data and plot it:

In [ ]:
# Set seed for reproducibility
set.seed(123)

# Generate synthetic time series
seq_len <- 1000  # Length of the sequence
x <- seq(1, seq_len)
data <- x/200 + sin(2*pi*x/50) + rnorm(seq_len, sd=0.5)

# Plot original data
plot(x, data, type="l", xlab="Time", ylab="Value", main="Synthetic Time Series")

Next, we prepare the data for model fitting

In [ ]:
window_size <- 10  # Consider 10 time steps to predict the next one
x_data <- NULL
y_data <- NULL

for (i in 1:(seq_len - window_size)) {
  x_data <- rbind(x_data, data[i:(i + window_size - 1)])
  y_data <- c(y_data, data[i + window_size])
}

# Reshape data for RNN
x_data <- array_reshape(x_data, c(dim(x_data), 1))
y_data <- array_reshape(y_data, c(length(y_data), 1))


Next, we define our RNN model. We'll use an LSTM layer which is usually better at capturing long-term dependencies than a simple RNN.

In [ ]:
model <- keras_model_sequential() %>%
  layer_lstm(units = 50, input_shape = list(window_size, 1)) %>%
  layer_dense(units = 1)

model %>% compile(
  optimizer = optimizer_adam(lr = 0.001),
  loss = "mean_squared_error"
)

summary(model)

### and train the model
history <- model %>% fit(
  x_data, y_data,
  epochs = 100, batch_size = 20,
  validation_split = 0.2,
  shuffle = TRUE
)

# Plotting loss
plot(history)


Finally, we check how well we can predict:

In [ ]:
pred_val <- model %>% predict(x_data)

# Plotting
par(mfrow = c(1, 1))
plot(x, data, type="l", xlab="Time", ylab="Value", main="Original and Predicted Time Series")
lines(11:1000, pred_val, col="red")